# Checking the No. of Streams

* Your first step always should be checking how many streams you have in the .xdf file.
* We use the pyxdf library to work with these files.
* The number should be $3$ in our case. If there are three streams then you can continue with other specific checks otherwise the recording was unsuccessful.

In [ ]:
import pyxdf
import pandas as pd
import numpy as np

streams, header = pyxdf.load_xdf('test_run1.xdf')

for s in streams:
    print(s['info']['name'][0])

# EEG Stream Check-Up

* Load the **streams**, change the name of the file and ideal if you put this script in the same folder as the files so you don't have to work through the directory related problems.
* Use `.head()` to check the 1st 5 values and `tail()` for the last 5 values to make sure the device was on during the whole time. The values of all channels should be non-zero.

In [ ]:
# Pick the Unicorn/Data stream as EEG
eeg = next(s for s in streams
           if s['info']['name'][0] == 'Unicorn')

# raw data matrix
data = eeg['time_series']

# rebuild timestamps if they’re flat or missing
if (eeg['time_stamps'] is None
    or np.allclose(np.diff(eeg['time_stamps']), 0)):
    try:
        srate = float(eeg['info']['nominal_srate'][0])
    except Exception:
        srate = 250.
    timestamps = np.arange(data.shape[0]) / srate
else:
    timestamps = eeg['time_stamps']

# channel labels (fallback to generic names)
try:
    chan_labels = [
        ch.decode()
        for ch in eeg['info']['desc'][0]
                             ['channels'][0]['channel']
    ]
except Exception:
    chan_labels = [f'ch{i}' for i in range(data.shape[1])]

# build DataFrame
eeg_df = pd.DataFrame(data, columns=chan_labels)
eeg_df.insert(0, 'time', timestamps)

print("EEG DataFrame:", eeg_df.shape)

eeg_df.tail()

EEG DataFrame: (397344, 18)


,time,ch0,ch1,ch2,ch3,ch4,ch5,ch6,ch7,ch8,ch9,ch10,ch11,ch12,ch13,ch14,ch15,ch16
397339,334675.608730,308500.65625,290122.43750,255868.125000,283525.71875,263405.03125,278643.37500,265150.53125,271883.50000,0.062256,0.987061,-0.159912,-1.251221,-1.220703,0.732422,60.0,659280.0,1.0
397340,334675.612727,308486.18750,289964.62500,255854.343750,283523.65625,263398.43750,278638.28125,264765.00000,271879.12500,0.061279,0.988525,-0.159424,-1.190185,-1.220703,0.854492,60.0,659281.0,1.0
397341,334675.616723,308482.25000,289941.75000,255853.093750,283515.71875,263395.65625,278638.28125,264754.09375,271876.62500,0.060547,0.989502,-0.157471,-1.251221,-1.098633,0.915527,60.0,659282.0,1.0
397342,334675.620720,308498.34375,290087.56250,255868.125000,283515.62500,263406.18750,278648.75000,265135.40625,271887.00000,0.060303,0.993164,-0.158203,-1.220703,-0.854492,0.762939,60.0,659283.0,1.0
397343,334675.624717,308506.75000,290196.28125,255876.796875,283517.93750,263414.96875,278656.43750,265382.09375,271892.09375,0.058838,0.995117,-0.156494,-1.098633,-0.793457,0.671387,60.0,659284.0,1.0


# Checking up the Markers Stream

In [34]:
# pick the marker stream by name
marker_streams = [s for s in streams if 'nBack_markers' in s['info']['name'][0]]
if not marker_streams:
    print("No marker stream found containing 'NBackMarkers'")
else:
    mkr = marker_streams[0]

# build a DataFrame of event times + labels
mkr_df = pd.DataFrame({
    'time'  : mkr['time_stamps'],
    'marker': [t[0] for t in mkr['time_series']]
})

# sort and inspect
mkr_df = mkr_df.sort_values('time').reset_index(drop=True)
print(f"Marker events:\n {mkr_df}")


Marker events:
                time                                             marker
0     333284.938593                                 main_block_0_start
1     333284.938657  sequence_D,H,R,P,H,Z,Q,K,E,H,H,K,M,U,H,H,Q,N,T...
2     333284.938681  targets_1,4,9,10,14,15,19,20,22,23,25,29,31,36...
3     333284.946777                            main_block_0_trial_0_on
4     333287.957966                                         resp_False
...             ...                                                ...
1484  334631.699265                                          corr_True
1485  334631.699271                                       trial_47_end
1486  334631.699277                                   main_block_6_end
1487  334631.699287                         main_block_6_accuracy_0.88
1488  334634.226278                                         resp_False

[1489 rows x 2 columns]


# Checking up the Mic Stream

In [19]:
# Pick the audio stream by name (e.g., 'MicDecibels')
audio = next(s for s in streams if s['info']['name'][0] == 'MicDecibels')

# Build a DataFrame of audio dB values and timestamps
audio_df = pd.DataFrame({
    'time': audio['time_stamps'],
    'decibels': [row[0] for row in audio['time_series']]
})

# Sort and inspect
audio_df = audio_df.sort_values('time').reset_index(drop=True)
print("Audio dB events:")
print(audio_df.tail())

Audio dB events:
                time   decibels
68385  334675.430232 -64.047272
68386  334675.479909 -65.495430
68387  334675.529601 -49.848625
68388  334675.569920 -56.870735
68389  334675.619745 -67.059242
